In [39]:
import pyautogui as gui
import pywinauto as wind
import pytesseract
from time import sleep
from pywinauto.application import Application
import path_location
import re

pytesseract.pytesseract.tesseract_cmd = path_location.TESSERACT_LOC
RST_LOCATION = path_location.RST_LOCATION

In [40]:
def click_here(x=None, y=None, pause=None, debug=False):
    gui.moveTo(x, y, pause=pause)
    if debug:
        gui.press('ctrl', presses=2, interval=.7)
    gui.click()

def get_app_prop(app):
    app_prop = app.rectangle()
    return (app_prop.left, app_prop.top, app_prop.width(), app_prop.height())

def get_app_prop2(app):
    app_prop = app.client_rect()
    return app_prop.width(), app_prop.height()

def get_sreenshot(app=None, wind_stat=None):
    if app != None:
        app.set_focus()
        app_prop = get_app_prop(app)
        wind_stat = (app_prop[0], app_prop[1], app_prop[2], app_prop[3])
    elif wind_stat == None:
        print('[ERROR] Either app object or custom box has to be passed')
        return 
    return gui.screenshot(region=wind_stat)

def image_convert(image):
    greyscale = image.convert('L')
    blackwhite = greyscale.point(lambda x: 0 if x < 200 else 255, '1')
    return image
    # return blackwhite

In [41]:
def app_get_region(app, x, y, width=10, height=10, use_ratio=False, draw_outline=True):
    # PCARS 2 options change based on windows size (resolution)
    if use_ratio:
        app_h = app.client_rect().height()
        app_w = app.client_rect().width()
        x = x * app_w/2560
        y = y * app_h/1440
        width = width * app_w/2560
        height = height * app_h/1440
    x,y,width,height = int(x),int(y),int(width),int(height)
    x, y = app.client_to_screen((x,y))
    
    if draw_outline:
        class cusrect:
            left=x
            top=y
            right=left+width
            bottom=top+height

        for _ in range(25):
            app.draw_outline(rect=cusrect)
    
    return x, y, width, height

In [42]:
def ocr(img, v1=False, raw=False):
    if v1:
        return pytesseract.image_to_string(img).split('\n')
    else:
        import io
        import json
        import cv2
        import numpy as np
        import requests

        img.save("screenshot.jpg")
        img = cv2.imread("screenshot.jpg")
        url_api = "https://api.ocr.space/parse/image"
        _, compressedimage = cv2.imencode(".jpg", img, [1, 90])
        file_bytes = io.BytesIO(compressedimage)

        result = requests.post(url_api,
                    files = {"screenshot.jpg": file_bytes},
                    data = {"apikey": path_location.OCR_SPACE_API_KEY,
                            "language": "eng",'isOverlayRequired':raw})
        result = result.content.decode()
        result = json.loads(result)
        if raw:
            return result
        parsed_results = result.get("ParsedResults")[0]
        return parsed_results.get("ParsedText").split()

In [43]:
try:
    pc2 = Application().connect(title_re='.*Project CARS 2.*')['Project Cars 2']
except wind.findwindows.ElementNotFoundError:
    print('[ERROR] Launch PCARS 2 first')
#     quit()
    
try:
    rst = Application().connect(title_re='MainWindow').MainWindow
except wind.findwindows.ElementNotFoundError:
    rst = Application().start(RST_LOCATION+'\\RST Software.exe').MainWindow
except wind.findwindows.ElementAmbiguousError:
    print('[ERROR] Sad, but for now, close the params child window.')
#     quit()

D:\Users\GV\Anaconda3\lib\site-packages\pywinauto\application.py:1087: UserWarning: 32-bit application should be automated using 32-bit Python (you use 64-bit Python)
  UserWarning)


## RST

In [44]:
def open_rst_params():
    rst.set_focus()
    RST_FILE_OFFSET_X = 350
    RST_FILE_OFFSET_Y = 10
    RST_NEXT_OFFSET_X = RST_FILE_OFFSET_X+10
    RST_NEXT_OFFSET_Y = RST_FILE_OFFSET_Y+75

    #Select file
    x, y, width, height = app_get_region(rst, RST_FILE_OFFSET_X, RST_FILE_OFFSET_Y, 60, 40, draw_outline=True)
    click_here(x=x+width/2, y=y+height/2, pause=0.5)
    
    #Select next car parameters
    x, y, width, height = app_get_region(rst, RST_NEXT_OFFSET_X, RST_NEXT_OFFSET_Y, 280, 30, draw_outline=True)
    click_here(x=x+width/2, y=y+height/2, pause=0.5)


def find_rst_settings(pos):
    setup_bar_width = get_app_prop2(rst)[0]
    mid_op = setup_bar_width/4
    return (int(mid_op*pos), int(setup_bar_width/4))

def open_rst_settings(settings):
    settings_type = {'suspension':1, 'dampers':2, 'gearing':3}
    rst.set_focus()
    x, width = find_rst_settings(settings_type[settings])
    x, y, width, height = app_get_region(rst, x, 70, width, 40, draw_outline=True)
    click_here(x+width/2, y+height/2, pause=.5, debug=True)

open_rst_params()

## PCARS2

In [45]:
def get_stats_gear_pc2():
    # Select ECU menu
    rst.set_focus()
    pc2.set_focus()
    pos = app_get_region(pc2, x=int(get_app_prop2(pc2)[0]/2)+300, y=340,  width=10, height=10, use_ratio=True)
    gui.click(pos[0], pos[1])

    x, y, width, height = app_get_region(pc2, x=1370, y=730,  width=100, height=320, use_ratio=True)
    content = image_convert(get_sreenshot(wind_stat=(x, y, width, height)))
    return ocr(content)

def set_stats_gear_rst(gears):
    rst.set_focus()
    rst_pos_x, rst_pos_y = rst.rectangle().left, rst.rectangle().top
    rst_pos_height, rst_pos_width = rst.rectangle().height(), rst.rectangle().width()
    rst_width_mid = rst_pos_x+rst_pos_width/2
    rst_y_firstfield = rst_pos_y+180
    for gear, val in enumerate(gears):
        gui.click((rst_width_mid, rst_y_firstfield+55*gear))
        try:
            float(val)
            gui.typewrite(val)
        except ValueError:
            print(f'[ERROR] Gear {gear} ({val}) not a number')
            gui.typewrite('0')

gears = get_stats_gear_pc2()
open_rst_settings('gearing')
set_stats_gear_rst(gears)

In [46]:
def retriev_ocr(content, debug=True):
    results = ocr(content)
    content = []
    if len(results) == 0:
        content.append('0')
    else:
        for val in results:
            try:
                content.append(re.match('\\d+', val).group())
            except:
                content.append('0')
    if debug:
        # content.show()
        print(results)
        print(content)
    return content

In [47]:
def get_stats_sus_pc2():
    # Select suspension menu
    rst.set_focus()
    pc2.set_focus()
    pos = app_get_region(pc2, x=int(get_app_prop2(pc2)[0]/2)-300, y=340, use_ratio=True)
    gui.click(pos[0], pos[1])

    # Get front spring
    x, y, width, height = app_get_region(pc2, x=865, y=700,  width=130, height=40, use_ratio=True)
    content = image_convert(get_sreenshot(wind_stat=(x, y, width, height)))
    f_spring = retriev_ocr(content)[0]

    # Get rear spring
    x, y, width, height = app_get_region(pc2, x=865, y=1050,  width=130, height=40, use_ratio=True)
    content = image_convert(get_sreenshot(wind_stat=(x, y, width, height)))
    r_spring = retriev_ocr(content)[0]
    
    # Get front antirollbar
    x, y, width, height = app_get_region(pc2, x=1350, y=665,  width=140, height=40, use_ratio=True)
    content = image_convert(get_sreenshot(wind_stat=(x, y, width, height)))
    f_ar = retriev_ocr(content)[0]
    
    # Get rear antirollbar
    x, y, width, height = app_get_region(pc2, x=1350, y=1010,  width=140, height=40, use_ratio=True)
    content = image_convert(get_sreenshot(wind_stat=(x, y, width, height)))
    r_ar = retriev_ocr(content)[0]
    
    return f_spring, r_spring, f_ar, r_ar

def set_stats_sus_rst(stats):
    rst.set_focus()
    
    prop= get_app_prop2(rst)
    seperator = prop[0]/3
    start = seperator/2

    pos = app_get_region(rst, x=start, y=230)
    gui.click(pos[0], pos[1])
    gui.typewrite(stats[0])

    pos = app_get_region(rst, x=start+seperator, y=230)
    gui.click(pos[0], pos[1])
    gui.typewrite(stats[2])

    pos = app_get_region(rst, x=start, y=435)
    gui.click(pos[0], pos[1])
    gui.typewrite(stats[1])

    pos = app_get_region(rst, x=start+seperator, y=435)
    gui.click(pos[0], pos[1])
    gui.typewrite(stats[3])

In [48]:
stats = get_stats_sus_pc2()
open_rst_settings('suspension')
set_stats_sus_rst(stats)

[]
['0']
[]
['0']
[]
['0']


KeyboardInterrupt: 

In [36]:
def get_stats_damp_pc2():
    # Select dampers menu
    rst.set_focus()
    pc2.set_focus()
    pos = app_get_region(pc2, x=int(get_app_prop2(pc2)[0]/2), y=340, use_ratio=True)
    gui.click(pos[0], pos[1])

    # Get front damp
    x, y, width, height = app_get_region(pc2, x=860, y=580,  width=140, height=280, use_ratio=True)
    content = image_convert(get_sreenshot(wind_stat=(x, y, width, height)))
    f_damp = retriev_ocr(content)

    # Get rear damp
    x, y, width, height = app_get_region(pc2, x=860, y=965,  width=140, height=280, use_ratio=True)
    content = image_convert(get_sreenshot(wind_stat=(x, y, width, height)))
    r_damp = retriev_ocr(content)
    
    return f_damp, r_damp

def set_stats_sus_rst(stats):
    rst.set_focus()
    
    prop= get_app_prop2(rst)
    seperator = prop[0]/3
    start = seperator/2

    pos = app_get_region(rst, x=start, y=230)
    gui.click(pos[0], pos[1])
    gui.typewrite(stats[0])

    pos = app_get_region(rst, x=start+seperator, y=230)
    gui.click(pos[0], pos[1])
    gui.typewrite(stats[2])

    pos = app_get_region(rst, x=start, y=435)
    gui.click(pos[0], pos[1])
    gui.typewrite(stats[1])

    pos = app_get_region(rst, x=start+seperator, y=435)
    gui.click(pos[0], pos[1])
    gui.typewrite(stats[3])

In [37]:
get_stats_damp_pc2()

['7000N/m/s', '1050N/m/s', '3050N/m/s', '53011m/s', '38N', '-59N']
['7000', '1050', '3050', '53011', '38', '0']
['IOOOON/m/s', '14500N/m/s', '4000N/m/s', '7450N/m/s', '501', '-770N']
['0', '14500', '4000', '7450', '501', '0']


(['7000', '1050', '3050', '53011', '38', '0'],
 ['0', '14500', '4000', '7450', '501', '0'])

In [38]:
x, y, width, height = app_get_region(pc2, x=860, y=580,  width=140, height=280, use_ratio=True)
x, y, width, height = app_get_region(pc2, x=860, y=965,  width=140, height=280, use_ratio=True)

MatchError: Could not find 'Project Cars 2' in 'dict_keys([])'

In [49]:
pc2.client_rect()

<RECT L0, T0, R1768, B992>

In [53]:
x, y, width, height = app_get_region(pc2, x=865, y=700,  width=130, height=40, use_ratio=True)

In [54]:
x

993